#### Import all import lib

In [5]:
import os
import warnings
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

warnings.filterwarnings("ignore")

load_dotenv()

True

#### OpenAI APIs initialization

In [6]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### Initialise the OpenAI model

In [7]:
model = OpenAI(
    temperature=0.9,
    model_name = "gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

#### Source code download

In [ ]:
from git import Repo

source_code = "../source_code"
url = "https://github.com/atikul-islam-sajib/ESRGAN"

Repo.clone_from(url=url, to_path=source_code)